In [1]:
import os
import subprocess
import shutil
import fileinput

import rasterio
import geopandas as gpd
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import fiona
import rasterio
from rasterio.mask import mask
from rasterio.mask import raster_geometry_mask
from osgeo import gdal

# Run LSDTT command-line script to get channel heads

includes making driver files for each tile. 'test.driver' was a driver file I made ahead of time. I will probably go back and tune some of that.

This will take a while. It will also appear in your terminal. 

In [ ]:
for fname in os.listdir("."):
        if "_UTM.tif" in fname and "tile" in fname:
                position=fname.index('.')
                filename=fname[0:position]
                print("Getting channel heads on " + (os.path.join(".", fname)))
                
                #Need .bils. Shouldn't have deleted them! 
                gdal_string = "gdal_translate -of ENVI " + fname + " " + filename + ".bil"
                print(gdal_string)
                os.system(gdal_string)
                
                #Replace the driver file's read and write file name with the ucrrent filename
                shutil.copy("test.driver", filename+".driver")
                for line in fileinput.input(filename+".driver", inplace=True): 
                    print(line.rstrip().replace("read fname: fname", "read fname: " + filename))
                for line in fileinput.input(filename+".driver", inplace=True): 
                    print(line.rstrip().replace("write fname: fname", "write fname: " + filename))
                
                #Run channel head extraction command line script
                lsd_string="lsdtt-channel-extraction " + filename+".driver"
                os.system(lsd_string)

# Extract raster info at channel heads

This includes extracting drainage area at the points and building and checking against geologic masks. Only a minute or so, probably due to my shallow understanding of rasterio/Shapely

In [5]:
for fname in os.listdir("."):
        if "sources" in fname and "geol" not in fname and "channel" not in fname:
                csvname=fname
                position=fname.index('.')
                #I'm so sorry this is what I have to do
                #There must be a better way
                if fname[6]=="_":
                    tilename=fname[0:6]
                else:
                    tilename=fname[0:7]
                
                df=pd.read_csv(csvname)
                
                print(csvname)
                
                #Find the dinfinity raster that matches the csv
                for dinfname in os.listdir("."):
                    if tilename in dinfname and "dinf" in dinfname:
                        
                        dinf_src = rasterio.open(dinfname)
                
                        print("extracting " + dinfname + "from " + csvname )
                
                #get coordinates from the sources csv
                coords = [(x,y) for x, y in zip(df.x, df.y)]        
                df['dinf_area'] = [x[0] for x in dinf_src.sample(coords)]
                
                #definitely need to write a program for this. 
                gdem = gdal.Open(tilename + '_UTM.tif', gdal.GA_ReadOnly)
                proj = gdem.GetProjection()
                #So it looks like LSDTT might get the projection but I don't know how to tap that so I literally just
                #grab from this massive unformatted string the UTM zone from the gdal command
                proj=(proj[20:32])

                if '17N' in proj:
                    with fiona.open("Tuscarora_17N.shp", "r") as shapefile:
                        Tuscarora_mask = [feature["geometry"] for feature in shapefile]
                #    print('17N')
                elif '18N' in proj:
                    with fiona.open("Tuscarora_18N.shp", "r") as shapefile:
                        Tuscarora_mask = [feature["geometry"] for feature in shapefile]
                #    print('18N')
                else:
                    print('u r fucked') #I was very frustrated


                #Bless me father for I have sinned, this is the only way I could 
                #do a logical for whether a point is within the Tuscarora
                #Appears that Shapely should do it but it didn't like my shapefile being in 
                #a lot of pieces. I decided I'd rather get the answer the ugly way
                out_image, out_transform = rasterio.mask.mask(dinf_src, Tuscarora_mask, crop=True)
                out_meta = dinf_src.meta
                out_meta.update({"driver": "GTiff",
                                  "height": out_image.shape[1],
                                  "width": out_image.shape[2],
                                  "transform": out_transform})
                
                #Write a raster, close it, then open it...I'm sure this is not correct. 
                profile = dinf_src.profile       
                mask_ds= rasterio.open('mask_raster.tif', 'w', **profile)
                mask_ds.write(out_image[0], 1)
                mask_ds.close()
                masktest = rasterio.open('mask_raster.tif')
                
                #now you use pandas
                df['within_tusc'] = [x[0] for x in masktest.sample(coords)]
                df['within_tusc'] = np.where(df['within_tusc']< 0, False, True)
                df=df.to_csv(csvname+"_geol")

tile_0_UTM_ATsources.csv
extracting tile_0_dinf_area.tiffrom tile_0_UTM_ATsources.csv
tile_0_UTM_Dsources.csv
extracting tile_0_dinf_area.tiffrom tile_0_UTM_Dsources.csv
tile_0_UTM_Psources.csv
extracting tile_0_dinf_area.tiffrom tile_0_UTM_Psources.csv
tile_0_UTM_Wsources.csv
extracting tile_0_dinf_area.tiffrom tile_0_UTM_Wsources.csv
tile_10_UTM_ATsources.csv
extracting tile_10_dinf_area.tiffrom tile_10_UTM_ATsources.csv
tile_10_UTM_Dsources.csv
extracting tile_10_dinf_area.tiffrom tile_10_UTM_Dsources.csv
tile_10_UTM_Psources.csv
extracting tile_10_dinf_area.tiffrom tile_10_UTM_Psources.csv
tile_10_UTM_Wsources.csv
extracting tile_10_dinf_area.tiffrom tile_10_UTM_Wsources.csv
tile_11_UTM_ATsources.csv
extracting tile_11_dinf_area.tiffrom tile_11_UTM_ATsources.csv
tile_11_UTM_Dsources.csv
extracting tile_11_dinf_area.tiffrom tile_11_UTM_Dsources.csv
tile_11_UTM_Psources.csv
extracting tile_11_dinf_area.tiffrom tile_11_UTM_Psources.csv
tile_11_UTM_Wsources.csv
extracting tile_11_dinf_

tile_2_UTM_Psources.csv
extracting tile_20_dinf_area.tiffrom tile_2_UTM_Psources.csv
extracting tile_21_dinf_area.tiffrom tile_2_UTM_Psources.csv
extracting tile_22_dinf_area.tiffrom tile_2_UTM_Psources.csv
extracting tile_23_dinf_area.tiffrom tile_2_UTM_Psources.csv
extracting tile_24_dinf_area.tiffrom tile_2_UTM_Psources.csv
extracting tile_25_dinf_area.tiffrom tile_2_UTM_Psources.csv
extracting tile_26_dinf_area.tiffrom tile_2_UTM_Psources.csv
extracting tile_27_dinf_area.tiffrom tile_2_UTM_Psources.csv
extracting tile_28_dinf_area.tiffrom tile_2_UTM_Psources.csv
extracting tile_29_dinf_area.tiffrom tile_2_UTM_Psources.csv
extracting tile_2_dinf_area.tiffrom tile_2_UTM_Psources.csv
tile_2_UTM_Wsources.csv
extracting tile_20_dinf_area.tiffrom tile_2_UTM_Wsources.csv
extracting tile_21_dinf_area.tiffrom tile_2_UTM_Wsources.csv
extracting tile_22_dinf_area.tiffrom tile_2_UTM_Wsources.csv
extracting tile_23_dinf_area.tiffrom tile_2_UTM_Wsources.csv
extracting tile_24_dinf_area.tiffrom t

In [6]:
#Check your work. Sick. 
df2=pd.read_csv("tile_10_UTM_Wsources.csv_geol")
df2.head()

,Unnamed: 0,id,x,y,latitude,longitude,dinf_area,within_tusc
0,0,31979,729635.25,4372377.5,39.470364,-78.330492,1276.491333,False
1,1,72523,728234.25,4372278.5,39.469846,-78.346796,311.441254,True
2,2,109784,730652.25,4372212.5,39.468607,-78.318738,29896.201172,False
3,3,116598,728861.25,4372200.5,39.468977,-78.339541,17690.310547,True
4,4,139385,726200.25,4372164.5,39.469357,-78.370456,583.874451,False


# Create new csv files with tile means of channel head drainage areas

In [7]:
#I make a csv for each channel head algorithm. W is the best, last time I looked at the data. 
df3=pd.DataFrame()
for fname in os.listdir("."):
        if "geol" in fname and "Wsources" in fname:
            
            #every time I run these lines a kitten dies
            if fname[6]=="_":
                tilename=fname[0:6]
            else:
                tilename=fname[0:7]
                        
            df2=pd.read_csv(fname)
            t_m = df2['dinf_area'][df2['within_tusc']==True].mean()
            nt_m = df2['dinf_area'][df2['within_tusc']==False].mean()
            t_s = df2['dinf_area'][df2['within_tusc']==True].std()
            nt_s = df2['dinf_area'][df2['within_tusc']==False].std()
            data = [{'tile': tilename,'stype': "Wsources",'tusc_mean':t_m, 'ntusc_mean':nt_m, 'tusc_std':t_s, 'ntusc_std':nt_s}]
            df3_0=pd.DataFrame(data)
            df3=df3.append(df3_0)
            
df3.to_csv('Wsources_channel_heads.csv', index=False)

In [8]:
df3

,tile,stype,tusc_mean,ntusc_mean,tusc_std,ntusc_std
0,tile_0,Wsources,11562.723354,9203.488593,3399.926652,15502.876722
0,tile_10,Wsources,9178.533895,19793.466347,6233.959199,106868.482556
0,tile_11,Wsources,9926.139148,12431.761606,4923.803107,18007.934792
0,tile_13,Wsources,13199.304464,12400.762581,18266.924634,51314.867000
0,tile_14,Wsources,17390.382717,10662.500630,25566.849261,25403.660649
0,tile_16,Wsources,8273.264114,12551.226222,5864.698326,24865.945771
0,tile_17,Wsources,24678.757810,12900.100321,65265.937408,14519.030288
0,tile_18,Wsources,7700.233411,12575.189017,5834.103981,26875.337584
0,tile_19,Wsources,10262.769982,14513.800571,4541.756762,27803.347849
0,tile_20,Wsources,12785.227536,10967.146747,22973.917341,29949.345862


In [9]:
df3=pd.DataFrame()
for fname in os.listdir("."):
        if "geol" in fname and "Dsources" in fname:

            if fname[6]=="_":
                tilename=fname[0:6]
            else:
                tilename=fname[0:7]
                        
            df2=pd.read_csv(fname)
            t_m = df2['dinf_area'][df2['within_tusc']==True].mean()
            nt_m = df2['dinf_area'][df2['within_tusc']==False].mean()
            t_s = df2['dinf_area'][df2['within_tusc']==True].std()
            nt_s = df2['dinf_area'][df2['within_tusc']==False].std()
            data = [{'tile': tilename,'stype': "Dsources",'tusc_mean':t_m, 'ntusc_mean':nt_m, 'tusc_std':t_s, 'ntusc_std':nt_s}]
            df3_0=pd.DataFrame(data)
            df3=df3.append(df3_0)
            #plt.scatter(ntusc_mean, df2['latitude'].mean(), c="black")
            #plt.scatter(tusc_mean, df2['latitude'].mean(), c="blue")
            
df3.to_csv('Dsources_channel_heads.csv', index=False)

In [10]:
df3=pd.DataFrame()
for fname in os.listdir("."):
        if "geol" in fname and "Psources" in fname:

            if fname[6]=="_":
                tilename=fname[0:6]
            else:
                tilename=fname[0:7]
                        
            df2=pd.read_csv(fname)
            t_m = df2['dinf_area'][df2['within_tusc']==True].mean()
            nt_m = df2['dinf_area'][df2['within_tusc']==False].mean()
            t_s = df2['dinf_area'][df2['within_tusc']==True].std()
            nt_s = df2['dinf_area'][df2['within_tusc']==False].std()
            data = [{'tile': tilename,'stype': "Psources",'tusc_mean':t_m, 'ntusc_mean':nt_m, 'tusc_std':t_s, 'ntusc_std':nt_s}]
            df3_0=pd.DataFrame(data)
            df3=df3.append(df3_0)
            #plt.scatter(ntusc_mean, df2['latitude'].mean(), c="black")
            #plt.scatter(tusc_mean, df2['latitude'].mean(), c="blue")
            
df3.to_csv('Psources_channel_heads.csv', index=False)

In [11]:
df3=pd.DataFrame()
for fname in os.listdir("."):
        if "geol" in fname and "ATsources" in fname:

            if fname[6]=="_":
                tilename=fname[0:6]
            else:
                tilename=fname[0:7]
                        
            df2=pd.read_csv(fname)
            t_m = df2['dinf_area'][df2['within_tusc']==True].mean()
            nt_m = df2['dinf_area'][df2['within_tusc']==False].mean()
            t_s = df2['dinf_area'][df2['within_tusc']==True].std()
            nt_s = df2['dinf_area'][df2['within_tusc']==False].std()
            data = [{'tile': tilename,'stype': "ATsources",'tusc_mean':t_m, 'ntusc_mean':nt_m, 'tusc_std':t_s, 'ntusc_std':nt_s}]
            df3_0=pd.DataFrame(data)
            df3=df3.append(df3_0)
            #plt.scatter(ntusc_mean, df2['latitude'].mean(), c="black")
            #plt.scatter(tusc_mean, df2['latitude'].mean(), c="blue")
            
df3.to_csv('ATsources_channel_heads.csv', index=False)

Here there be some plotting scripts I'm holding on to in case I need them

In [ ]:
for fname in os.listdir("."):
        if "geol" in fname and "Wsources" in fname:
            df2=pd.read_csv(fname)
            tusc_mean = df2['dinf_area'][df2['within_tusc']==True].mean()
            ntusc_mean = df2['dinf_area'][df2['within_tusc']==False].mean()
            plt.scatter(df2['latitude'].mean(),tusc_mean-ntusc_mean,  c="black")
            

In [ ]:
for fname in os.listdir("."):
        if "geol" in fname and "Dsources" in fname:
            df2=pd.read_csv(fname)
            tusc_mean = df2['dinf_area'].mean()
            plt.scatter(df2['latitude'].mean(),tusc_mean,  c="blue")
            

In [ ]:
for fname in os.listdir("."):
        if "geol" in fname and "Dsources" in fname:
            df2=pd.read_csv(fname)
            tusc_mean = df2['dinf_area'][df2['within_tusc']==True].mean()
            ntusc_mean = df2['dinf_area'][df2['within_tusc']==False].mean()
            plt.scatter(df2['latitude'].mean(),tusc_mean-ntusc_mean,  c="black")